# Sentimet Analysis

Demo tutorial for how to use nlp_toolkit to train classification model and predict new samples. The task we choose is sentiment binary classification.

The dataset is crawled from Kanzhun.com and Dajie.com, which is about company pros and cons.

Available models:
    1. Bi-LSTM Attention
    2. Multi Head Self Attention
    3. TextCNN

In [1]:
import sys
sys.path.append('../')
from nlp_toolkit import Dataset, Classifier

D:\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


## Data Processing

### Load config dict

In [2]:
import yaml
config = yaml.load(open('../nlp_toolkit/config.yaml'))

### Load Data

In [3]:
dataset = Dataset(fname='../data/company_pro_con.txt', task_type='classification', mode='train', segment=False, config=config)

2018-11-10 10:04:05,505 - data.py[line:80] - INFO: data loaded


In [4]:
for x, y in zip(dataset.texts[0:10], dataset.labels[0:10]):
    print(x, y)

进去 前 许诺 的 工资 给 的 高 0
校园 环境 优美 ， 美女 很多 ， 适合 居住 ， 食堂 饭菜 便宜 ， 操场 好 ， 可以 天天 运动 0
老板 人 很好 老 员工 会 各种 教 你 东西 ， 而且 不会 有所 保留 薪水 在 大连 还 算 可以 0
人员 比较 多 ， 复杂   办公室 容易 形成 拉帮结派 不利于 企业 发展 1
出差 太多 了 。 在 现场 开发 很苦 逼 。 1
公司 目前 地理位置 不 太 理想 ， 离 城市 中心 较 远点 。 1
公司 的 技术 水平 国内 顶尖 ， 十几 年 的 资历 ， 制作 的 作品 几乎 都 是 精品 ， 参与 过 很多 知名 项目 。 0
工作 流程 复杂     个人 上升 空间 有限     新产品 的 创新 能力 有限   组织 架构 稍 显 臃肿 1
无偿 加班 ， 加班 多 ， 没 加班费 ， 压力 很大 1
环境 比较 轻松 ， 跟 项目 走 ， 能 学 不少 专业 知识 ， 经验 很 重要 0


### Transform data to index

In [5]:
# if we want to use pre_trained embeddings, we need a gensim-format embedding file
x, y, config = dataset.transform()
print(x['word'].shape, y.shape)

2018-11-10 10:04:10,345 - data.py[line:107] - INFO: texts and labels transformed to number index
2018-11-10 10:04:15,013 - utilities.py[line:73] - INFO: OOV rate: 0.00 %
2018-11-10 10:04:15,028 - data.py[line:113] - INFO: Loaded Pre_trained Embeddings


(94635, 100) (94635, 2)


if your want to see the vocab and label index mapping dict

In [6]:
# dataset.transformer._word_vocab._token2id

In [7]:
# dataset.transformer._label_vocab._token2id

In [8]:
transformer = dataset.transformer

## Classifier Training

### Define classifier

In [9]:
model_name='bi_lstm_att'
# if you want to get attention weights during prediction, please set return_attention=True
# config[model_name]['return_att'] = True
text_classifier = Classifier(model_name=model_name, transformer=transformer, seq_type='bucket', config=config)

In [10]:
text_classifier.model

### Train Model

In [11]:
trained_model = text_classifier.train(x, y)

2018-11-10 10:04:17,472 - trainer.py[line:113] - INFO: bi_lstm_att model structure...
2018-11-10 10:04:17,497 - trainer.py[line:123] - INFO: train/valid set: 75708/18927
2018-11-10 10:04:17,497 - trainer.py[line:80] - INFO: use bucket sequence to speed up model training
2018-11-10 10:04:17,500 - sequence.py[line:300] - INFO: Training with 99 non-empty buckets


__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
word (InputLayer)               (None, None)         0                                            
__________________________________________________________________________________________________
embedding (Embedding)           (None, None, 300)    7333200     word[0][0]                       
__________________________________________________________________________________________________
activation_1 (Activation)       (None, None, 300)    0           embedding[0][0]                  
__________________________________________________________________________________________________
embed_drop (SpatialDropout1D)   (None, None, 300)    0           activation_1[0][0]               
__________________________________________________________________________________________________
bi_lstm_0 

2018-11-10 10:04:17,758 - sequence.py[line:300] - INFO: Training with 99 non-empty buckets


True
mointor training process using f1 score
Successfully made a directory: models\bi_lstm_att_201811101004
using Early Stopping
using Reduce LR On Plateau
tracking loss history and metrics


2018-11-10 10:04:18,065 - trainer.py[line:154] - INFO: saving model parameters and transformer to models\bi_lstm_att_201811101004


model hyperparameters:
 {'nb_classes': 2, 'nb_tokens': 24444, 'maxlen': None, 'embedding_dim': 300, 'rnn_size': 512, 'attention_dim': 128, 'embed_dropout_rate': 0.25, 'final_dropout_rate': 0.5, 'return_attention': True}
Epoch 1/25
1234/1234 [==============================] - 123s 100ms/step - loss: 0.4488 - acc: 0.5412 - val_loss: 0.2648 - val_acc: 0.6760
0 - f1: 93.37
1 - f1: 92.92

Epoch 00001: f1 improved from -inf to 0.93158, saving model to models\bi_lstm_att_201811101004\model_weights_01_0.6760_0.9316.h5
Epoch 2/25
1234/1234 [==============================] - 118s 96ms/step - loss: 0.2377 - acc: 0.6440 - val_loss: 0.2227 - val_acc: 0.7041
0 - f1: 94.33
1 - f1: 93.89

Epoch 00002: f1 improved from 0.93158 to 0.94121, saving model to models\bi_lstm_att_201811101004\model_weights_02_0.7041_0.9412.h5
Epoch 3/25
1234/1234 [==============================] - 119s 96ms/step - loss: 0.1765 - acc: 0.7844 - val_loss: 0.2153 - val_acc: 0.7877
0 - f1: 94.59
1 - f1: 94.09

Epoch 00003: f1 impr

### 10-fold training

In [14]:
config['train']['mode'] = 'fold'
text_classifier = Classifier(model_name=model_name, transformer=transformer, seq_type='bucket', config=config)
text_classifier.train(x, y)

2018-11-10 10:47:20,204 - trainer.py[line:113] - INFO: bi_lstm_att model structure...
2018-11-10 10:47:20,233 - trainer.py[line:123] - INFO: train/valid set: 75708/18927
2018-11-10 10:47:20,234 - trainer.py[line:80] - INFO: use bucket sequence to speed up model training
2018-11-10 10:47:20,236 - sequence.py[line:300] - INFO: Training with 99 non-empty buckets


__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
word (InputLayer)               (None, None)         0                                            
__________________________________________________________________________________________________
embedding (Embedding)           (None, None, 300)    7333200     word[0][0]                       
__________________________________________________________________________________________________
activation_2 (Activation)       (None, None, 300)    0           embedding[0][0]                  
__________________________________________________________________________________________________
embed_drop (SpatialDropout1D)   (None, None, 300)    0           activation_2[0][0]               
__________________________________________________________________________________________________
bi_lstm_0 

2018-11-10 10:47:20,495 - sequence.py[line:300] - INFO: Training with 99 non-empty buckets


True
mointor training process using f1 score
Successfully made a directory: models\bi_lstm_att_201811101047
using Early Stopping
using Reduce LR On Plateau
tracking loss history and metrics


2018-11-10 10:47:20,815 - trainer.py[line:154] - INFO: saving model parameters and transformer to models\bi_lstm_att_201811101047


model hyperparameters:
 {'nb_classes': 2, 'nb_tokens': 24444, 'maxlen': None, 'embedding_dim': 300, 'rnn_size': 512, 'attention_dim': 128, 'embed_dropout_rate': 0.25, 'final_dropout_rate': 0.5, 'return_attention': True}
Epoch 1/25
  68/1233 [>.............................] - ETA: 3:31 - loss: 1.2162 - acc: 0.6667

KeyboardInterrupt: 

## Predict New Samples

## Load data and transformer

In [ ]:
dataset = Dataset('../data/company_pro_con_predict.txt',
                  task_type='classification', mode='predict',
                  tran_fname='models/bi_lstm_att_201811101004/transformer.h5',
                  segment=False)
x_seq, n_labels = dataset.transform()

## Load model

In [ ]:
text_classifier = Classifier('multi_head_self_att', dataset.transformer)
text_classifier.load(weight_fname='models/bi_lstm_att_201811101004/model_weights_03_0.7877_0.9435.h5',
                     para_fname='models/bi_lstm_att_201811101004/model_parameters.json')

## predict samples

In [ ]:
y_pred = text_classifier.predict(x_seq['word'], n_labels)

### attention visualization

In [ ]:
model_name='bi_lstm_att'
config[model_name]['return_att'] = True

text_classifier = Classifier(model_name=model_name, mode='train', transformer=transformer, config=new_config)
